In [1]:
import ants
import tifffile
import numpy as np
import os
# import cv2
import csv
import gc
import json
import time
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from scipy import stats
from scipy.ndimage import zoom
import scipy.ndimage as ndi

%matplotlib inline
import matplotlib.pyplot as plt

from multiprocessing import Pool
plt.gray()

import json, os.path, os, re, time
import joblib
import subprocess as sp
import nibabel as nib
import scipy.spatial
import cupy as cp
import cupyx.scipy.ndimage as ndi
# cp.array(0)
import PointDetection as cd

ModuleNotFoundError: No module named 'PointDetection_CPU'

<Figure size 640x480 with 0 Axes>

In [ ]:
np.arange(1)

In [ ]:
def Gaussian_filter_GPU(img_o, sigma):
    img = cp.asarray(img_o)
    dst=cpn.gaussian_filter(img,sigma=sigma, mode="reflect")
    dst2=cp.asnumpy(dst)
    del img
    del dst
    cp.get_default_memory_pool().free_all_blocks()
    return dst2

def min_max(x:np.ndarray):
    minval = x.min(axis=None, keepdims=True)
    maxval= x.max(axis=None, keepdims=True)
    result = (x-minval)/(maxval-minval)
    return result    
    
def make_psf(size, sigma):
    p = np.zeros((size, size, size))
    p[size//2, size//2, size//2] = 1
    p = ndimage.gaussian_filter(p, sigma)
    p /= p.sum()
    return p
    
def LR_deconv(img,psf,i):
    im=img.copy()
    for n in range(i):
        print(n)
        normalize_and_convert_to_float(im)
        im1=im.copy()
        conv1=img/ndimage.convolve(im1, psf, mode='constant', cval=0.0)
        conv2=ndimage.convolve(conv1, psf, mode='constant', cval=0.0)
        im=im1*conv2 
    return im

def normalize_and_convert_to_float(array):
    float_array = array.astype(np.float32)
    min_val = float_array.min()
    max_val = float_array.max()
    normalized_array = (float_array - min_val) / (max_val - min_val)
    return normalized_array

   
def LR_deconv_GPU(img, psf, i, zval=40, overlap=10,zoom=(2.5, 2.5, 2.5)):
    psf=normalize_and_convert_to_float(psf)
    psf_cp =  cp.asarray(psf).astype(cp.float32)   
    im = img.astype(cp.float32)
    result = np.empty_like(im)
    for start_z in range(0, im.shape[0], zval - overlap):
        print("\tProcess deconvolution:"+str(start_z))
        end_z = min(start_z + zval, im.shape[0])
        im_chunk =im[start_z:end_z]
        im_chunk_gpu=cp.asarray(im_chunk).astype(cp.float32)
        im_chunk_gpu_d=im_chunk_gpu.copy()
        for n in range(i):
            conv1 = cp.divide(im_chunk_gpu_d, ndi.convolve(im_chunk_gpu, psf_cp, mode='constant', cval=0.0))#
            conv2 = ndi.convolve(conv1, psf_cp, mode='constant', cval=0.0)#
            if(n!=(i-1)):
                im_chunk_gpu *= conv2#u(t+1)
        if start_z==0:
            result[start_z:end_z] = cp.asnumpy(im_chunk_gpu)
        else:
            result[start_z+overlap//2:end_z] = cp.asnumpy(im_chunk_gpu[overlap//2:])
        cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_memory_pool().free_all_blocks()
    return result



def read_tiff_stack(folder_path, start_index=0, num_images=-1):
    # Get the list of TIFF files
    
    tiff_files = [os.path.join(folder_path, filename)
                  for filename in os.listdir(folder_path)
                  if filename.lower().endswith(('.tif', '.tiff'))]
    # Sort the files to ensure correct order
    tiff_files.sort()

    # Select the specified range of files
    if num_images == -1:
        tiff_files = tiff_files[start_index:]
    else:
        tiff_files = tiff_files[start_index:(start_index + num_images)]
    print("Load tiff:" + str(start_index) + "-" + str(len(tiff_files)))
    first_tiff = tifffile.imread(tiff_files[0])
    shape = (len(tiff_files),) + first_tiff.shape
    stack = np.empty(shape, dtype=np.uint16)

    # Read each TIFF file and store it in the stack
    for i, tiff_file in enumerate(tiff_files):
        stack[i] = tifffile.imread(tiff_file)

    return stack


In [3]:
# cfos_fol="/home/gpu_data/data1/yamashitaData1/231012_circadian_2nd_Data1/231012_circadian_2nd_Reconst/"
# exp = "2nd"
cfos_fol="/home/gpu_data/data1/yamashitaData1/230828circadian_Data1/230828circadian_1st_Reconst/"
exp = "1st"
savedir = "/home/gpu_data/data8/cfos_app/"
Resize_um = 50

CT_li = np.arange(0,48,4)
sample_ids = np.arange(1,7,1)

reconsts = os.listdir(cfos_fol)
sample_names=[]
colors=[]
data_parent_paths=[]
data_moving_paths =[]

for CT in CT_li:
#     if CT>72:
#         continue
    for sample_id in sample_ids:
        sample = "CT"+str(CT)+"_"+str(sample_id).zfill(2)

        for reconst in reconsts:
#             sample_names.append(("_").join(reconst.split("_")[1:3]))

            if sample in reconst:
                sample_names.append(sample)
                for color in os.listdir(cfos_fol+reconst):
                    if "cfos" in color:
                        data_moving_paths.append(cfos_fol+reconst+"/"+color)
                    else:
                        data_parent_paths.append(cfos_fol+reconst+"/"+color)

print(len(data_parent_paths))   
print(len(data_moving_paths))
print(len(sample_names))

72
72
72


In [ ]:
#deconvolution

src ="/home/gpu_data/data7/"
dst = "/home/gpu_data/data8/results/"
pre_img_path = src+"/yamashita/202404_WT_SD_DD/" 
deconv_path = src +"/yamashita/202404_WT_SD_DD/deconv/" 
color_name = "cFos"      # Input directory name for the 1st color
os.makedirs(deconv_path, exist_ok=True)

psf=make_psf(9, (1.8,1.8,1.8))
z_chunk=100

c = 0
for fol in os.listdir(pre_img_path):
    if "CT0" in fol:
        continue
    if "Reconst" in fol:
        c += 1
        if c < 2:
            continue
        print(fol)
        for fol2 in os.listdir(pre_img_path + fol):
            if "cfos" in fol2:
                FPr = pre_img_path + fol +"/" + fol2
                FPw = deconv_path + fol +"/" + fol2
                if os.path.exists(FPw):
                    continue
                os.makedirs(FPw, exist_ok=True)
                
                
                
                zoom_factors=(2.5, 2.5, 2.5)
                overlap =psf.shape[0]+7
                overlap_zoom = int((psf.shape[0]) * 2)  # Adjust overlap based on zoom factor
                im = read_tiff_stack(FPr)
                im[im<1000]=1000
                os.makedirs(FPw, exist_ok=True)
                for start_z in range(0, im.shape[0], z_chunk - overlap // 2):
                    
                    print("Process z:" + str(start_z))
                    end_z = min(start_z + z_chunk, im.shape[0])
                    im2 = cp.asnumpy(cpx_ndimage.zoom(cp.asarray(im[start_z:end_z] // 2), zoom=zoom_factors, order=3))
                    cp.get_default_memory_pool().free_all_blocks()
                    RC = LR_deconv_GPU(im2, psf, 10, zval=40, overlap=overlap_zoom)
                    del im2
                    gc.collect()
                    cp.get_default_memory_pool().free_all_blocks()
                    RC = np.clip(RC * 2, 0, 65535).astype(np.uint16)
                    
                    for i, z_slice in enumerate(RC):
                        if start_z != 0 and i < overlap // 2:
                            continue
                        filename = f"{int(start_z*zoom_factors[0]+i):08d}.tif"
                        tifffile.imwrite(os.path.join(FPw, filename), z_slice)
                    del RC 
                    gc.collect()
                del im
                gc.collect()
    
        
    

In [ ]:
#detect cell candidates by intensity peaks
src = "/home/gpu_data/data1/"
dst = "/home/gpu_data/data7/"
root = src+"yamashitaData1/"

deconv_fs = ["230828circadian_Data1/230828circadian_1st_Reconst_median_norm_Deconv", "231012_circadian_2nd_Data1/231012_circadian_2nd_Reconst_median_norm_Deconv"]
postfix = "output_peak"
# FP = root+"/230828circadian_1st_Reconst_median_norm_Deconv"    # Input directory path for the parent folder
# FPout = root+"/cfos_1st_median_norm_Deconv_output"  # Input directory path for the output parent folder

structure_name = "SYTOX-G"  # Input directory name for structure image
color_name1 = "cfos"      # Input directory name for the 1st color
resolution = int(10/2.5)             # Input voxel size in micrometers (μm)

params=[5000, 0, 2500, -1]  #
peak_size = 7  #3
psf_sizes =  [13, 27] #[5, 11]
psf_sigmas = [(1,1,1), (2,1,1)]
line_sizes = [13, 25, 51]  #[5, 11, 23]
line_3d_sigmas = [(1.5,1.5,1.5)]
batchsize=100
overlap=10
threads=20
dirs=[]
cellnumber=[]
skip=1

for i, deconv_f in enumerate(deconv_fs):
    if i>0:
        continue
    
    FP = root+deconv_f    # Input directory path for the parent folder
    FPout =dst+deconv_f +"_"+postfix  # Input directory path for the output parent folder


    for d in os.listdir(FP):
        if(os.path.isdir(FP+"/"+d)):
            dirs.append(d)



    for dir in dirs:
#         if "CT0_01" in dir or "CT8_01" in dir or "CT16_01" in dir:
            if os.path.exists(FPout+"/"+dir+"/coordinates.csv"):
                print("{} already exist".format(FPout+"/"+dir+"/coordinates.csv"))
                continue
            print(FP+"/"+dir)
        #     sample_name = "CT0_01"
        #     if(sample_name in dir):
        #         skip=0
        #     if(skip==1):
        #         continue
            if(color_name1 in dir):
                FPr=FP+"/"+dir
                FPw=FPout+"/"+dir
                df=cd.GetCells(FPr, FPw, params, peak_size, psf_sizes, psf_sigmas, line_sizes, line_3d_sigmas, batchsize, overlap, threads)
                cellnumber.append((dir,str(len(df))))
#         skip=1
# with open(FPout+"/result.csv", "a", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(cellnumber)
        #break

In [2]:
#cell detect only peak deconv pre
src = "/home/gpu_data/data1/"
dst = "/home/gpu_data/data7/"
root = src+"yamashitaData1/"

deconv_fs = ["230828circadian_Data1/230828circadian_1st_Reconst_median_norm_Deconv", "231012_circadian_2nd_Data1/231012_circadian_2nd_Reconst_median_norm_Deconv"]
postfix = "output_peak_2500"
# FP = root+"/230828circadian_1st_Reconst_median_norm_Deconv"    # Input directory path for the parent folder
# FPout = root+"/cfos_1st_median_norm_Deconv_output"  # Input directory path for the output parent folder
FP_PSF = src + "/matsumoto/CUBIC/spf"  # Input directory path to store PSF data
structure_name = "SYTOX-G"  # Input directory name for structure image
color_name1 = "cfos"      # Input directory name for the 1st color
resolution = int(10/2.5)             # Input voxel size in micrometers (μm)

params=[5000, 2500, 8.0, 1.5]
peak_size = 3
psf_sizes = [5, 11]
psf_sigmas = [(1,1,1), (2,1,1)]
line_sizes = [5, 11, 23]
line_3d_sigmas = [(1.5,1.5,1.5)]
batchsize=200
overlap=10
threads=15
dirs=[]
cellnumber=[]
skip=1

# for i, deconv_f in enumerate(deconv_fs):
#     if i>0:
#         continue
    
#     FP = root+deconv_f    # Input directory path for the parent folder
    
FPout =dst+ "/test_getcell_th" +"_"+postfix  # Input directory path for the output parent folder




FP = src+"/yamashitaData1/230828circadian_Data1/230828circadian_1st_Reconst/0066_CT0_01_Reconst/cfos_CT0_01'"

#     for dir in dirs:
#         if "CT0_01" in dir or "CT8_01" in dir or "CT16_01" in dir:

#             print(FP+"/"+dir)
    #     sample_name = "CT0_01"
    #     if(sample_name in dir):
    #         skip=0
    #     if(skip==1):
    #         continue

FPr=FP
FPw=FPout
df=cd.GetCells(FPr, FPw, params, peak_size, psf_sizes, psf_sigmas, line_sizes, line_3d_sigmas, batchsize, overlap, threads)
#             cellnumber.append((dir,str(len(df))))
#         skip=1
# with open(FPout+"/result.csv", "a", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(cellnumber)
        #break

27.5